# Esperimenti pratici con le API di Intelligenza Artificiale

Benvenuti alla seconda parte del nostro percorso sull'Intelligenza Artificiale! Dopo aver esplorato la teoria delle reti neurali e dei transformer, oggi metteremo in pratica ciò che abbiamo imparato utilizzando API disponibili gratuitamente.

In questo notebook esploreremo:
1. Come utilizzare le API di Hugging Face per accedere a modelli pre-addestrati
2. Come creare semplici interfacce grafiche con Gradio
3. Esempi pratici di applicazioni di AI

## Preparazione dell'ambiente

Per prima cosa, installiamo le librerie necessarie:

In [ ]:
# Installiamo le librerie necessarie
!pip install huggingface_hub requests gradio transformers

## 1. Utilizzo delle API di Hugging Face

Hugging Face offre un "Inference API" che permette di utilizzare modelli pre-addestrati senza dover gestire l'infrastruttura. È possibile utilizzare questa API gratuitamente con alcuni limiti di utilizzo.

Per prima cosa, vediamo come funziona una semplice richiesta API utilizzando la libreria `requests`:

In [ ]:
import requests

# URL dell'API di inferenza di Hugging Face
API_URL = "https://api-inference.huggingface.co/models/gpt2"

# È possibile utilizzare un token personale per aumentare i limiti
# Se non lo avete, potete utilizzare l'API senza token con limiti più restrittivi
# Per ottenere un token: https://huggingface.co/settings/tokens
API_TOKEN = ""  # Lasciate vuoto se non avete un token

headers = {}
if API_TOKEN:
    headers["Authorization"] = f"Bearer {API_TOKEN}"

# Funzione per fare una richiesta al modello
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Esempio di generazione di testo con GPT-2
input_text = "L'intelligenza artificiale è"
output = query({"inputs": input_text})

print(f"Input: {input_text}")
print(f"Output: {output[0]['generated_text']}")

### Proviamo con diversi modelli

Hugging Face offre una vasta gamma di modelli per diversi compiti. Vediamo alcuni esempi:

In [ ]:
# Funzione più generica per utilizzare diversi modelli
def query_model(model_name, inputs, task="text-generation"):
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {}
    if API_TOKEN:
        headers["Authorization"] = f"Bearer {API_TOKEN}"
    
    payload = {"inputs": inputs}
    
    # Opzioni aggiuntive per la generazione di testo
    if task == "text-generation":
        payload["parameters"] = {"max_length": b0, "temperature": 0.7}
    
    response = requests.post(api_url, headers=headers, json=payload)
    return response.json()

## 2. Classificazione di testo

Proviamo un modello di classificazione delle emozioni:

In [ ]:
# Modello per l'analisi del sentimento
sentiment_model = "distilbert-base-uncased-finetuned-sst-2-english"

# Testo da analizzare
texts = [
    "I love artificial intelligence!",
    "This class is boring.",
    "I'm not sure how I feel about this."
]

for text in texts:
    result = query_model(sentiment_model, text, task="sentiment-analysis")
    print(f"Testo: {text}")
    print(f"Risultato: {result}\n")

## 3. Traduzione automatica

Vediamo come utilizzare un modello di traduzione:

In [ ]:
# Modello di traduzione
translation_model = "Helsinki-NLP/opus-mt-en-it"

# Testi da tradurre
english_texts = [
    "Artificial intelligence is changing the world.",
    "Today we will learn how to use AI APIs.",
    "Programming is fun and creative."
]

for text in english_texts:
    result = query_model(translation_model, text, task="translation")
    print(f"Inglese: {text}")
    print(f"Italiano: {result[0]['translation_text']}\n")

## 4. Sintesi di testo (Summarization)

Vediamo come utilizzare un modello per riassumere testi lunghi:

In [ ]:
# Modello di summarization
summarization_model = "facebook/bart-large-cnn"

# Testo lungo da riassumere
long_text = """
L'intelligenza artificiale (IA) è un campo dell'informatica che si occupa della creazione di macchine capaci di pensare e agire come esseri umani. Negli ultimi anni, l'IA ha fatto progressi significativi grazie all'aumento della potenza di calcolo, alla disponibilità di grandi quantità di dati e allo sviluppo di algoritmi più sofisticati. Le applicazioni dell'IA spaziano dalla medicina alla finanza, dal trasporto all'intrattenimento.

Un sottocampo particolarmente attivo dell'IA è l'apprendimento automatico (machine learning), in cui i sistemi imparano dai dati senza essere esplicitamente programmati. Il deep learning, un sottoinsieme del machine learning basato su reti neurali artificiali con molti strati (deep), ha rivoluzionato settori come la visione artificiale, l'elaborazione del linguaggio naturale e la traduzione automatica.

I modelli transformer, introdotti nel 2017 con il paper "Attention is All You Need", rappresentano una svolta fondamentale nel campo dell'elaborazione del linguaggio naturale. Questi modelli utilizzano un meccanismo di attenzione per comprendere le relazioni tra le parole in un testo, migliorando significativamente le prestazioni in compiti come la traduzione, la generazione di testo e la risposta a domande.
"""

result = query_model(summarization_model, long_text, task="summarization")
print("Testo originale:\n", long_text)
print("\nRiassunto:\n", result[0]['summary_text'])

## 5. Riconoscimento di immagini

Vediamo come utilizzare un modello per riconoscere oggetti in un'immagine.
Per questo esempio, useremo un'immagine da URL:

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

# URL di un'immagine (in questo caso, un'immagine di un cane)
image_url = "https://images.unsplash.com/photo-1517849845537-4d257902454a"

# Scarica l'immagine
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Mostra l'immagine
plt.figure(figsize=(10, 8))
plt.imshow(image)
plt.axis('off')
plt.show()

# Converte l'immagine in bytes
with BytesIO() as buffer:
    image.save(buffer, format="JPEG")
    image_bytes = buffer.getvalue()

# Modello per la classificazione di immagini
vision_model = "google/vit-base-patch16-224"

# URL dell'API per la visione
vision_api_url = f"https://api-inference.huggingface.co/models/{vision_model}"

# Fai la richiesta API
headers = {}
if API_TOKEN:
    headers["Authorization"] = f"Bearer {API_TOKEN}"
response = requests.post(vision_api_url, headers=headers, data=image_bytes)
result = response.json()

# Mostra i risultati
print("L'immagine contiene probabilmente:")
for item in result:
    print(f"{item['label']}: {item['score']*100:.2f}%")

## 6. Creazione di un'interfaccia con Gradio

Ora creiamo un'interfaccia grafica per interagire con i modelli in modo più intuitivo. Utilizzeremo Gradio, che è una libreria Python che permette di creare rapidamente interfacce per modelli di machine learning.

In [ ]:
import gradio as gr

# Funzione che utilizza il modello di generazione di testo
def generate_text(prompt, model_name="gpt2", max_length=100):
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {}
    if API_TOKEN:
        headers["Authorization"] = f"Bearer {API_TOKEN}"
    
    payload = {
        "inputs": prompt,
        "parameters": {
            "max_length": max_length,
            "temperature": 0.7
        }
    }
    
    response = requests.post(api_url, headers=headers, json=payload)
    result = response.json()
    
    if isinstance(result, list) and len(result) > 0 and 'generated_text' in result[0]:
        return result[0]['generated_text']
    return str(result)  # In caso di errore, restituisce l'errore come stringa

# Creazione dell'interfaccia Gradio per la generazione di testo
demo_text = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=2, placeholder="Inserisci un testo iniziale..."),
        gr.Dropdown(["gpt2", "distilgpt2", "gpt2-medium"], label="Modello", value="gpt2"),
        gr.Slider(50, 200, value=100, step=10, label="Lunghezza massima")
    ],
    outputs="text",
    title="Generatore di Testo con AI",
    description="Inserisci un testo iniziale e il modello completerà il resto."
)

# Funzione che utilizza il modello di traduzione
def translate_text(text, source_lang, target_lang):
    # Determina il modello in base alle lingue selezionate
    model_name = f"Helsinki-NLP/opus-mt-{source_lang}-{target_lang}"
    
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    headers = {}
    if API_TOKEN:
        headers["Authorization"] = f"Bearer {API_TOKEN}"
    
    payload = {"inputs": text}
    
    try:
        response = requests.post(api_url, headers=headers, json=payload)
        result = response.json()
        
        if isinstance(result, list) and len(result) > 0 and 'translation_text' in result[0]:
            return result[0]['translation_text']
        return str(result)  # In caso di errore, restituisce l'errore come stringa
    except Exception as e:
        return f"Errore: {str(e)}. Verifica che il modello per la coppia di lingue {source_lang}-{target_lang} esista."

# Creazione dell'interfaccia Gradio per la traduzione
demo_translation = gr.Interface(
    fn=translate_text,
    inputs=[
        gr.Textbox(lines=4, placeholder="Inserisci un testo da tradurre..."),
        gr.Dropdown(["en", "it", "fr", "es", "de"], label="Lingua di origine", value="en"),
        gr.Dropdown(["it", "en", "fr", "es", "de"], label="Lingua di destinazione", value="it")
    ],
    outputs="text",
    title="Traduttore con AI",
    description="Inserisci un testo e seleziona le lingue per tradurlo."
)

# Funzione per l'analisi del sentimento
def analyze_sentiment(text):
    model_name = "distilbert-base-uncased-finetuned-sst-2-english"
    api_url = f"https://api-inference.huggingface.co/models/{model_name}"
    
    headers = {}
    if API_TOKEN:
        headers["Authorization"] = f"Bearer {API_TOKEN}"
    
    payload = {"inputs": text}
    
    try:
        response = requests.post(api_url, headers=headers, json=payload)
        result = response.json()
        
        if isinstance(result, list) and len(result) > 0 and 'label' in result[0]:
            sentiment = result[0]['label']
            score = result[0]['score']
            return f"Sentimento: {sentiment}\nConfidenza: {score*100:.2f}%"
        return str(result)
    except Exception as e:
        return f"Errore: {str(e)}"

# Creazione dell'interfaccia Gradio per l'analisi del sentimento
demo_sentiment = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=2, placeholder="Inserisci un testo in inglese per analizzare il sentimento..."),
    outputs="text",
    title="Analisi del Sentimento con AI",
    description="Inserisci un testo in inglese per determinare se esprime un sentimento positivo o negativo."
)

# Unisci tutte le demo in un'unica interfaccia con schede
demo = gr.TabbedInterface(
    [demo_text, demo_translation, demo_sentiment],
    ["Generazione di Testo", "Traduzione", "Analisi del Sentimento"],
    title="Esperimenti con le API di AI"
)

# Avvia l'interfaccia
demo.launch(debug=True)

## 7. Bonus: Utilizzo della libreria Transformers di Hugging Face

Finora abbiamo utilizzato le API remote di Hugging Face. Se avete abbastanza potenza di calcolo (idealmente una GPU), potete anche eseguire i modelli localmente utilizzando la libreria `transformers`.

Ecco un esempio di come utilizzare un modello di generazione di testo localmente:

In [ ]:
from transformers import pipeline

# Crea un pipeline di generazione di testo
# Nota: quando eseguite questo codice per la prima volta, il modello verrà scaricato
# (può richiedere un po' di tempo e spazio su disco)
text_generator = pipeline('text-generation', model='distilgpt2')

# Genera testo
result = text_generator("L'intelligenza artificiale è", max_length=50, do_sample=True, temperature=0.7)
print(result[0]['generated_text'])

## Esercizi per gli studenti

1. **Esercizio base**: Modifica il prompt nel generatore di testo e osserva come cambia l'output.
2. **Esercizio intermedio**: Crea un'interfaccia Gradio per il modello di riassunto di testo.
3. **Esercizio avanzato**: Integra un modello di generazione di immagini come Stable Diffusion utilizzando le API di Hugging Face.

## Conclusioni

In questo notebook, abbiamo esplorato come utilizzare le API di Hugging Face per accedere a modelli di intelligenza artificiale e come creare semplici interfacce grafiche con Gradio. Questi strumenti rendono l'IA accessibile anche senza una grande potenza di calcolo o conoscenze approfondite.

Ricordate che l'utilizzo delle API di Hugging Face ha dei limiti nella versione gratuita, ma è sufficiente per scopi educativi e sperimentali.

## Risorse aggiuntive

- [Hugging Face - Inference API](https://huggingface.co/inference-api)
- [Documentazione di Gradio](https://www.gradio.app/docs/)
- [Hugging Face - Transformers](https://huggingface.co/docs/transformers/index)
- [Hugging Face Spaces](https://huggingface.co/spaces) - Esempi di applicazioni create con Gradio e Streamlit